In [5]:
import os,sys,mlflow
sys.path.append(os.path.abspath(".."))

from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample,models,losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator,BinaryClassificationEvaluator
from sentence_transformers.readers import STSBenchmarkDataReader

from sklearn.model_selection import train_test_split

from  mlflow.tracking import MlflowClient
from torch.utils.data import DataLoader
from modules import utils
import torch
import torch.nn as nn
from torch.autograd import Variable

import torch,re,time,math,logging,os,mlflow
import pandas as pd
import numpy as np

#logger = utils.init_logger(__name__) # setup logger

%reload_ext utils

TOP_N = 12000
TRAIN_N = 100


df = utils.load_text_data().head(TRAIN_N).drop_duplicates(subset=["text"])
df.shape

/Users/tomasztkaczyk/Desktop/axel/ci-topic-thunder/code/utils.py:62: DeprecationWarning: invalid escape sequence \]
  news = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), '', news)
/Users/tomasztkaczyk/Desktop/axel/ci-topic-thunder/code/utils.py:64: DeprecationWarning: invalid escape sequence \d
  news = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", news)
/Users/tomasztkaczyk/Desktop/axel/ci-topic-thunder/code/utils.py:65: DeprecationWarning: invalid escape sequence \S
  news = re.sub('http\S+\s*', '', news)  # remove URLs
/Users/tomasztkaczyk/Desktop/axel/ci-topic-thunder/code/utils.py:83: DeprecationWarning: invalid escape sequence \]
  news = re.sub('[{}]'.format(re.escape("""▶︎►…!"#$%&'(!)*+,/:;<=>?@[\]^_`‚‘{|}~""")), '', news)
/Users/tomasztkaczyk/Desktop/axel/ci-topic-thunder/code/utils.py:84: DeprecationWarning: invalid escape sequence \d
  news = re.sub("\d.","",news)
/Users/tomasztkaczyk/Desktop/axel/ci-topic-thunder/code/utils.py:91: DeprecationWarning: invalid es

Number of stopwords 607


(99, 8)

In [6]:


remote_server="http://ec2-18-192-179-113.eu-central-1.compute.amazonaws.com"
os.environ["MLFLOW_S3_ENDPOINT_URL"]="http://s3.eu-central-1.amazonaws.com"
mlflow.set_experiment("Fine Tuning SBERT - 15.10")


INFO: 'Fine Tuning SBERT - 15.10' does not exist. Creating a new experiment


### Load data and prepapre the input for the model

In [7]:
headlines=utils.preprocess_text(df,col="headline")
seo_titles=utils.preprocess_text(df,col="seo_title")

pairs = [[a,b] for (a,b) in zip(headlines,seo_titles) if len(a)>3 and len(b)> 3 ]

headlines_train, headlines_test, seo_titles_train, seo_titles_test \
    = train_test_split(np.array(pairs)[:,0], np.array(pairs)[:,1], test_size=0.2, random_state=1)

In [13]:
def train_config(examples_a,examples_b, model, batch_size):
    '''train dataloader and model.'''
    print(f"Read  train dataset")
    train_examples = [InputExample(texts=[a,b],label=float(1)) for (a,b) in zip(examples_a,examples_b)]

    train_data = SentencesDataset(train_examples, model)
    train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)
    train_evaluator =  BinaryClassificationEvaluator.from_input_examples(train_examples)
    return train_data, train_loss, train_dataloader, train_evaluator

def dev_config(examples_a,examples_b, model, batch_size):
    '''dev dataloader and model'''
    print(f"Read dev dataset")
    dev_examples = [InputExample(texts=[a,b],label=1) for (a,b) in zip(examples_a,examples_b)]

    dev_data = SentencesDataset(dev_examples, model=model)
    dev_dataloader = DataLoader(dev_data, shuffle=False, batch_size=batch_size)
    dev_loss = losses.CosineSimilarityLoss(model=model)
    dev_evaluator = BinaryClassificationEvaluator.from_input_examples(dev_examples)
    return dev_loss, dev_dataloader, dev_evaluator

# Initialize model

In [14]:
# load BERT model from Hugging Face
word_embedding_model = models.Transformer(
    'T-Systems-onsite/bert-german-dbmdz-uncased-sentence-stsb')

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [15]:
batch_size =32


train_data, train_loss, train_dataloader, train_evaluator = train_config(headlines_train,seo_titles_train,model,batch_size)
dev_loss, dev_dataloader, dev_evaluator = dev_config(headlines_test,seo_titles_test,model,batch_size)

Read  train dataset
Read dev dataset


In [16]:
num_epochs=4
warmup_steps = math.ceil(len(headlines_train) * num_epochs / batch_size * 0.1) #10% of train data for warm-up

model.fit(train_objectives=[(train_dataloader, train_loss)],
            evaluator=train_evaluator,
          epochs=num_epochs,use_amp=True,
          evaluation_steps=100,
          warmup_steps=warmup_steps,
          output_path="./_fine-tune-bert-german-dbmdz-uncased-sentence-BILD-100k",
         #callback= callback_fn 
         )

with mlflow.start_run():
    mlflow.log_param("num_epochs",num_epochs)
    mlflow.log_param("warmup_steps",warmup_steps)
    mlflow.log_param("batch_size",warmup_steps)
    mlflow.log_param("train_size",len(headlines_train))
    mlflow.log_param("eval_size",len(headlines_test))
    
    def callback_fn(s,epoch,steps):
        print("callback")
        mlflow.log_metric("score",s,steps)
        mlflow.log_metric("score_epoch",s,epoch)
        

    

/Users/tomasztkaczyk/opt/anaconda3/envs/py38/lib/python3.8/site-packages/torch/cuda/amp/autocast_mode.py:114: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")


/Users/tomasztkaczyk/opt/anaconda3/envs/py38/lib/python3.8/site-packages/torch/cuda/amp/autocast_mode.py:114: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")


/Users/tomasztkaczyk/opt/anaconda3/envs/py38/lib/python3.8/site-packages/torch/cuda/amp/autocast_mode.py:114: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")
